In [1]:
import tensorflow as tf
import sklearn

2023-03-05 01:18:13.946442: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 01:18:14.059465: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-05 01:18:14.059488: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-05 01:18:14.600981: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


2023-03-05 01:18:25.053998: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-05 01:18:25.054025: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-05 01:18:25.054043: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (munna0912-VivoBook-ASUSLaptop-X421IAY-M413IA): /proc/driver/nvidia/version does not exist
2023-03-05 01:18:25.054297: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


In [4]:
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
BATCH_SIZE

16

In [4]:
Batch_Size = BATCH_SIZE

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from nltk.util import ngrams
import itertools
N_for_NGram = 3
Sequence_length = 200
epochs = 10
n_classes = 2
# Define a function to preprocess the link
def preprocess_link(link):
    
    # Convert all letters to lowercase
    link = link.lower()
    
    link = link.replace("http://","")
    link = link.replace("https://","")
    link = link.replace("www.","")
    link = link.replace(" ","")
    link = link.strip()
    
    # Remove punctuation and special characters
    link = re.sub('[^A-Za-z0-9]+', '', link)
    
    return link
from sklearn.model_selection import train_test_split
url_dataframe = pd.read_csv("../dataset/df_final.csv")
url_dataframe['url'] = url_dataframe['url'].apply(preprocess_link)
url_dataframe = url_dataframe.drop_duplicates(subset=['url'])
url_dataframe = url_dataframe.dropna().reset_index(drop=True)

import tensorflow as tf
train_df, test_df = train_test_split(url_dataframe, test_size=0.20)
steps_per_epoch = len(train_df)// BATCH_SIZE
validation_steps = len(test_df)// BATCH_SIZE



In [ ]:
max_tokens = 5000
train_ds = tf.data.Dataset.from_tensor_slices((train_df['url'].values, train_df['type'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['url'].values, test_df['type'].values))
train_ds = train_ds.batch(Batch_Size)

Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                   output_mode='int',
                                                   max_tokens = max_tokens,
                                                   output_sequence_length=Sequence_length)

# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x, y: x)
Vectorize_Layer.adapt(train_text)
train_ds = train_ds.unbatch()
train_ds = train_ds.shuffle(10000)
train_ds = train_ds.repeat()
train_ds = train_ds.batch(Batch_Size)
test_ds = test_ds.batch(Batch_Size)

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return Vectorize_Layer(text), label

train_ds = train_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)
with strategy.scope():
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_tokens, 64, input_length=Sequence_length),
    tf.keras.layers.LSTM(64, return_sequences=True) ,
    tf.keras.layers.LSTM(64,) ,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')])

    print(model.summary())

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], )

history = model.fit(
    train_ds,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch)

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Define the evaluation function
def evaluate_model(model, test_dataset):
    y_true = []
    for x, y in test_dataset:
        for yi in y:
            y_true.append(yi.numpy())
    
    y_true = np.array(y_true)
    y_true = y_true.flatten()
    y_pred = model.predict(test_dataset)
    y_pred = y_pred.flatten()
    y_pred = [round(i) for i in y_pred]
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Accuracy
    acc = accuracy_score(y_true, y_pred)
    print("Accuracy:", acc)

    # Precision
    prec = precision_score(y_true, y_pred, average='weighted')
    print("Precision:", prec)

    # Recall
    rec = recall_score(y_true, y_pred, average='weighted')
    print("Recall:", rec)

    # F1 Score
    f1 = f1_score(y_true, y_pred, average='weighted')
    print("F1 Score:", f1)
    
    return [cm,acc,prec,rec,f1]
# Evaluate the model
result = evaluate_model(model, test_ds)

import pickle
with open("3_gram_200_5k_model.pkl", "wb") as f:
    pickle.dump(model, f)
    
with open("3_gram_200_5k_result.pkl", "wb") as f:
    pickle.dump(result, f)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 64)           320000    
                                                                 
 lstm (LSTM)                 (None, 200, 64)           33024     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                     

2023-03-05 01:16:07.294984: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "UnbatchDataset/_7"
op: "UnbatchDataset"
input: "MapDataset/_6"
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\020UnbatchDataset:6"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
      shape {
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_STRING
      type: DT_INT64
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_STRING
        }
      }
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT

Epoch 1/10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:0').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:0').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0